## OBJECTIVE: Create a chatbot using the ChatGPT API that interacts with a given website URL. Although no frontend is required, the chatbot must be demonstrable via console.

In [1]:
# @title Installing libraries

!pip install openai 
import openai
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import os

!pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


!pip install langchain
from transformers import pipeline, BertForQuestionAnswering # Replacing Conversion with BertForQuestionAnswering as an example
from langchain.text_splitter import RecursiveCharacterTextSplitter # splitting text into chunks
!pip install gradio

import gradio as gr

ERROR: Invalid requirement: '#'


     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     ------- ----------------------------- 30.7/149.4 kB 660.6 kB/s eta 0:00:01
     --------------- --------------------- 61.4/149.4 kB 656.4 kB/s eta 0:00:01
     ---------------------- -------------- 92.2/149.4 kB 581.0 kB/s eta 0:00:01
     ----------------------------- ------ 122.9/149.4 kB 602.4 kB/s eta 0:00:01
     ---------------------------------- - 143.4/149.4 kB 607.9 kB/s eta 0:00:01
     ------------------------------------ 149.4/149.4 kB 523.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/18.1 MB 1.7 MB/s eta 0:00:11
   ---------------------------------------- 0.1/18.1 MB 1.1 MB/s eta 0:00:18
   ---------------------------------------- 0.2/18.1 MB 1.3 MB/s eta 0:00:14
   ---

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.9.2 which is incompatible.


ImportError: cannot import name 'GetCoreSchemaHandler' from 'pydantic' (C:\Users\Lenovo\anaconda3\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [2]:
# Step 1: Web scraping
url = 'https://botpenguin.com/'
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    website_content = ' '.join([p.text for p in paragraphs])

In [3]:
website_content

'Why BotPenguin   Product   Solutions   Partners   Resources   Why BotPenguin   Product   Solutions   Pricing   Partners   Resources  Integrations Experience 80+ world-class integrations. Key Features Take your business to the next level with our awesome key features. Live Chat Stay in the loop with your clientele 24*7! Unified Inbox Serve your customers across all platforms. Analytics Speedtrack your growth with our top-of-the-line analytics tools! Mobile App Make, monitor, & manage your AI chatbots with our mobile app. CHATBOT COMPARISONS WHAT CAN IT DO? Marketing Automation Make marketing a boon from the automation gods! FB Automation Engage with your customers on a deeper level. WhatsApp Automation Get that nifty automation for WhatsApp too! Appointment Bookings No more delays, BotPenguin’s got you here! Customer Support Your customers are in for a treat with this automation. Lead Generation Gain more lead without any extra effort or expenses WHO CAN USE IT? Healthcare Give your pa

In [4]:
# Check for non-alphanumeric characters
non_alphanumeric_chars = re.findall(r'[^a-zA-Z0-9\s]', website_content)
print("Non-alphanumeric characters found:", non_alphanumeric_chars)

# Check for extra whitespaces
extra_whitespaces = re.findall(r'\s+', website_content)
print("Extra whitespaces found:", extra_whitespaces)


Non-alphanumeric characters found: ['+', '-', '.', '.', '*', '!', '.', '-', '-', '-', '!', ',', ',', '&', '.', '?', '!', '.', '!', ',', '’', '!', '.', '?', '-', '!', '!', '-', '-', '!', '!', '-', '!', '!', '&', '!', '-', ',', ',', '&', '.', '?', ',', ',', '/', '.', '!', '-', '.', '.', '.', '.', '.', '’', '-', '.', '.', ',', '-', ',', ',', '-', ',', '’', ',', '!', ',', ',', '&', ',', ',', '.', '.', '-', '-', '!', '.', '&', '?', '’', '&', ',', ',', ',', '&', '%', '%', '%', '+', '+', '+', '+', '’', '.', '’', '.', ',', '.', '.', '.', ',', ',', '.', ',', ',', ',', '.', ',', ',', '.', '’', '-', '.', ',', ',', '-', '-', ',', ',', ',', ',', ',', ',', '‘', '’', '.', '.', '-', '-', '-', '.', '.', ',', '.', '.', ',', '.', ',', '.', '-', ',', '.', '.', ',', '&', '!', '.', ',', '.', ',', '&', '.', '+', '.', '’', ',', ',', '’', '.', '.', '+', ',', '.', '.', ',', '.', '"', '.', '.', '-', ',', '.', '"', '-', '-', '"', ',', '.', '.', '"', '-', '.', '.', '-', '"', '&', '.', '.', '"', '-', '-', '"', '.',

In [5]:
# Remove extra whitespaces and non-alphanumeric characters
website_content = re.sub(r'[^a-zA-Z0-9\s]', '', website_content)
website_content = re.sub(r'\s+', ' ', website_content)
website_content = website_content.strip()

In [6]:
website_content

'Why BotPenguin Product Solutions Partners Resources Why BotPenguin Product Solutions Pricing Partners Resources Integrations Experience 80 worldclass integrations Key Features Take your business to the next level with our awesome key features Live Chat Stay in the loop with your clientele 247 Unified Inbox Serve your customers across all platforms Analytics Speedtrack your growth with our topoftheline analytics tools Mobile App Make monitor manage your AI chatbots with our mobile app CHATBOT COMPARISONS WHAT CAN IT DO Marketing Automation Make marketing a boon from the automation gods FB Automation Engage with your customers on a deeper level WhatsApp Automation Get that nifty automation for WhatsApp too Appointment Bookings No more delays BotPenguins got you here Customer Support Your customers are in for a treat with this automation Lead Generation Gain more lead without any extra effort or expenses WHO CAN USE IT Healthcare Give your patients worldclass healthcare service Education

In [7]:
from langchain.schema import Document

# Initialize the RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Convert the website_content string to a Document object
documents = text_splitter.split_documents([Document(page_content=website_content)])
documents[:6]

[Document(page_content='Why BotPenguin Product Solutions Partners Resources Why BotPenguin Product Solutions Pricing Partners Resources Integrations Experience 80 worldclass integrations Key Features Take your business to the next level with our awesome key features Live Chat Stay in the loop with your clientele 247 Unified Inbox Serve your customers across all platforms Analytics Speedtrack your growth with our topoftheline analytics tools Mobile App Make monitor manage your AI chatbots with our mobile app CHATBOT COMPARISONS WHAT CAN IT DO Marketing Automation Make marketing a boon from the automation gods FB Automation Engage with your customers on a deeper level WhatsApp Automation Get that nifty automation for WhatsApp too Appointment Bookings No more delays BotPenguins got you here Customer Support Your customers are in for a treat with this automation Lead Generation Gain more lead without any extra effort or expenses WHO CAN USE IT Healthcare Give your patients worldclass healt

In [8]:
from transformers import pipeline

# Creating a pipeline for question answering & using a pre-trained model from Hugging Face
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2") # This model is deepset/roberta-base-squad2


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForQuestionAnswering: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForQuestionAnswering for predictions without further training.
C:\Users\Lenovo\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `

In [13]:
def chatbot_response(user_question):
  """
  This function takes a user's question and returns a response
  based on the website content using the QA pipeline.
  """

  best_answer = qa_pipeline({
      'question': user_question,
      'context': website_content
  })

  return best_answer['answer']


In [ ]:
user_input = input("You: ")

response = chatbot_response(user_input)
print("Chatbot:", response)

You:  what cost it charges


In [11]:
user_input = input("You: ")

response = chatbot_response(user_input)
print("Chatbot:", response)

You: What features does BotPenguin offer?
Chatbot: advanced services and features


In [12]:
user_input = input("You: ")

response = chatbot_response(user_input)
print("Chatbot:", response)

You: What features are included in the free plan?
Chatbot: Endless Integrations


In [13]:
user_input = input("You: ")

response = chatbot_response(user_input)
print("Chatbot:", response)

You: how many chatbots made with BotPenguin?
Chatbot: 100 FREE


In [14]:
user_input = input("You: ")

response = chatbot_response(user_input)
print("Chatbot:", response)

You: How does BotPenguin work?
Chatbot: responding to Facebook messages automatically


In [16]:
# run this in gradioo

def chatbot_interface(user_question):
  """
  This function takes a user's question and returns a response
  based on the website content using the QA pipeline.
  """

  best_answer = qa_pipeline({
      'question': user_question,
      'context': website_content
  })

  return best_answer['answer']


iface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="BotPenguin Chatbot",
    description="Ask me anything about BotPenguin",
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://04378ee52fd895fd01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
